# 05: Sentiment Analyser

> "Discover the power of Artificial Intelligence with OpenAI's powerful API to generate text and images in projects". Udemy course.

client id: i3-iwzp1ScZpgu7_Zruelg

client secret: cri4X8ZP9Eg_lFgdDW7TD18JjpojSw

In [91]:
client_id = 'i3-iwzp1ScZpgu7_Zruelg'
client_secret = 'cri4X8ZP9Eg_lFgdDW7TD18JjpojSw'

In [92]:
import openai 
import praw
import os

In [93]:
reddit = praw.Reddit(client_id=client_id, client_secret=client_secret, user_agent='Sentiment Analysis')

In [94]:
subr = reddit.subreddit('football')
subr.accounts_active

84

In [95]:
for post in subr.hot(limit=5):
    print(post.title)
    submission = reddit.submission(id=post.id)
    # print top 2 comments por title
    count = 0 
    for comment in submission.comments:
        if comment.body == '[deleted]':
            continue 
        print('>> ' + comment.body)
        count += 1
        if count == 2:
            break

/r/Football Daily Discussion Thread
**Important Update for r/football - Elevating Content Quality!*
>> Thankyou!

Would be nice to give them fanboys megathread really. Ronaldo vs Messi megathread so they can be contained, at least for the next 2-3 years they'll still talk about their deities.
>> Thank god
Dani Alves released from jail after paying €1m bail and serving a quarter of his sentence for rape
>> Wow, only one quarter of his sentence. That’s shocking
>> The law has always been a minor inconvenience to those with money.
Why is there a lack of successful English managers in football?
>> because it is the richest league so they just export the best which deprive english managers from big opprutunities. 

Also, English dont invest in managers like germany. Germany has a formal education for them and produce so many more than english produced

I dont think it is wrong if a NT aquire a foreign coach, so english should do that, because fukcing southgate is a clown and failure, and ev

In [96]:
def get_titles(subr='football', limit=6, num_comm=3, skip=2):
    subr = reddit.subreddit(subr)
    title_comm = {}
    
    for counter, post in enumerate(subr.hot(limit=limit)):
        if counter < skip:
            continue
        counter += (1-skip)
        title_comm[counter] = ''
        submission = reddit.submission(id=post.id)
        title = post.title

        title_comm[counter] += f'Title: {title} \n\n'
        title_comm[counter] += 'Comments: \n\n'
        comm_count = 0
        for comment in submission.comments:
            if not comment.body == '[deleted]':
                title_comm[counter] += f'>> {comment.body} \n'
                comm_count += 1
            if comm_count == num_comm:
                break
    return title_comm 

In [97]:
title_and_comments = get_titles()
title_and_comments

{1: 'Title: Dani Alves released from jail after paying €1m bail and serving a quarter of his sentence for rape \n\nComments: \n\n>> Wow, only one quarter of his sentence. That’s shocking \n>> The law has always been a minor inconvenience to those with money. \n>> How can you be released on bail if you have already been sentenced, Isnt bail only applicable if your awaiting trial? \n',
 2: 'Title: Why is there a lack of successful English managers in football? \n\nComments: \n\n>> because it is the richest league so they just export the best which deprive english managers from big opprutunities. \n\nAlso, English dont invest in managers like germany. Germany has a formal education for them and produce so many more than english produced\n\nI dont think it is wrong if a NT aquire a foreign coach, so english should do that, because fukcing southgate is a clown and failure, and even when he went to euro final and wc quarter he had easy path, so he always lose from teams of same level as engl

In [98]:
def create_prompt(title_and_comments):
    prompt = 'Return the sentiment of the following posts (positive, negative or neutral): \n\n'
    return prompt + title_and_comments

In [99]:
prompt = create_prompt(title_and_comments[1])
prompt

'Return the sentiment of the following posts (positive, negative or neutral): \n\nTitle: Dani Alves released from jail after paying €1m bail and serving a quarter of his sentence for rape \n\nComments: \n\n>> Wow, only one quarter of his sentence. That’s shocking \n>> The law has always been a minor inconvenience to those with money. \n>> How can you be released on bail if you have already been sentenced, Isnt bail only applicable if your awaiting trial? \n'

## Gpt analysis

In [100]:
#os.environ["OPENAI_API_KEY"] = "sk-"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [101]:
title_and_comments.items()

dict_items([(1, 'Title: Dani Alves released from jail after paying €1m bail and serving a quarter of his sentence for rape \n\nComments: \n\n>> Wow, only one quarter of his sentence. That’s shocking \n>> The law has always been a minor inconvenience to those with money. \n>> How can you be released on bail if you have already been sentenced, Isnt bail only applicable if your awaiting trial? \n'), (2, 'Title: Why is there a lack of successful English managers in football? \n\nComments: \n\n>> because it is the richest league so they just export the best which deprive english managers from big opprutunities. \n\nAlso, English dont invest in managers like germany. Germany has a formal education for them and produce so many more than english produced\n\nI dont think it is wrong if a NT aquire a foreign coach, so english should do that, because fukcing southgate is a clown and failure, and even when he went to euro final and wc quarter he had easy path, so he always lose from teams of same 

In [103]:
for key, title_with_comments in title_and_comments.items():
    prompt = create_prompt(title_with_comments)
    
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ],
        temperature=0,
        max_tokens=256,
        top_p=1,
    )
    print(f'Title {key}\n{title_with_comments} \nsentiment: {response.choices[0].message.content}')
    print('-'*40)

Title 1
Title: Dani Alves released from jail after paying €1m bail and serving a quarter of his sentence for rape 

Comments: 

>> Wow, only one quarter of his sentence. That’s shocking 
>> The law has always been a minor inconvenience to those with money. 
>> How can you be released on bail if you have already been sentenced, Isnt bail only applicable if your awaiting trial? 
 
sentiment: Negative
----------------------------------------
Title 2
Title: Why is there a lack of successful English managers in football? 

Comments: 

>> because it is the richest league so they just export the best which deprive english managers from big opprutunities. 

Also, English dont invest in managers like germany. Germany has a formal education for them and produce so many more than english produced

I dont think it is wrong if a NT aquire a foreign coach, so english should do that, because fukcing southgate is a clown and failure, and even when he went to euro final and wc quarter he had easy path,